In [1]:
import os
from utils import *
import optuna

In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val=split_data2(data, 0.2)

In [3]:
rat_rank_study = optuna.create_study(
    study_name="rating+ranking",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-06 19:46:47,825] A new study created in memory with name: rating+ranking


In [4]:
slim_el_study = optuna.create_study(
    study_name="slim_el",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-06 19:46:47,851] A new study created in memory with name: slim_el


In [5]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

slim_el_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(data_train, verbose=False)
slim_el_recommender.fit(alpha=0.002085488964546896, 
                topK=303, 
                l1_ratio=0.0026626042226214576)

#slimelastic [alpha: 0.002085488964546896, topK: 303, l1_ratio: 0.0026626042226214576]

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
rp3beta_study = optuna.create_study(
    study_name="P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

rp3beta_recommender = RP3betaRecommender(data_train, verbose=False)
rp3beta_recommender.fit(topK=35, alpha=0.3598110596257423, beta=0.1558338653772352, implicit=True, normalize_similarity=True)

#p3beta [topK: 35, alpha: 0.3598110596257423, beta: 0.1558338653772352, implicit: True, normalize_similarity: True]

In [ ]:
user_id=50

In [ ]:
slim_el_score = slim_el_recommender._compute_item_score(user_id).flatten()
slim_el_score

In [ ]:
rp3beta_score = rp3beta_recommender._compute_item_score(user_id).flatten()
rp3beta_score

In [ ]:
import seaborn as sns
from numpy import linalg as LA
import matplotlib.pyplot as plt

l1_slim_el = LA.norm([slim_el_score], 1)
l1_slim_el_scores = slim_el_score / l1_slim_el

l2_slim_el = LA.norm([slim_el_score], 2)
l2_slim_el_scores = slim_el_score / l2_slim_el

linf_slim_el = LA.norm(slim_el_score, np.inf)
linf_slim_el_scores = slim_el_score / linf_slim_el

lminusinf_slim_el = LA.norm(slim_el_score, -np.inf)
lminusinf_slim_el_scores = slim_el_score / lminusinf_slim_el

print(slim_el_score, l1_slim_el, l2_slim_el, linf_slim_el, lminusinf_slim_el)

_ = plt.figure(figsize=(16,7))
ax = sns.scatterplot(data=slim_el_score)
ax = sns.scatterplot(data=l1_slim_el_scores) 
ax = sns.scatterplot(data=l2_slim_el_scores) 
ax = sns.scatterplot(data=linf_slim_el_scores) 
ax = sns.scatterplot(data=lminusinf_slim_el_scores) 
ax.legend(["slim_el Score", "L1 Scores", "L2 Scores", "LInf Scores", "LMinusInf Scores"])

In [ ]:
l1_rp3beta = LA.norm(rp3beta_score, 1)
l1_rp3beta_scores = rp3beta_score / l1_rp3beta

l2_rp3beta = LA.norm(rp3beta_score, 2)
l2_rp3beta_scores = rp3beta_score / l2_rp3beta

linf_rp3beta = LA.norm(rp3beta_score, np.inf)
linf_rp3beta_scores = rp3beta_score / linf_rp3beta

lminusinf_rp3beta = LA.norm(rp3beta_score, -np.inf)
lminusinf_rp3beta_scores = rp3beta_score / lminusinf_rp3beta

print(rp3beta_score, l1_rp3beta, l2_rp3beta, linf_rp3beta, lminusinf_rp3beta)

_ = plt.figure(figsize=(16,9))
ax = sns.scatterplot(data=rp3beta_score)
ax = sns.scatterplot(data=l1_rp3beta_scores) 
ax = sns.scatterplot(data=l2_rp3beta_scores) 
ax = sns.scatterplot(data=linf_rp3beta_scores) 
ax = sns.scatterplot(data=lminusinf_rp3beta_scores) 
ax.legend(["rp3beta Score", "L1 Scores", "L2 Scores", "LInf Scores", "LMinusInf Scores"])

In [ ]:
lambda_weights = 0.66

l1_new_scores = lambda_weights * l1_rp3beta_scores + (1 - lambda_weights) * l1_slim_el_scores
l2_new_scores = lambda_weights * l2_rp3beta_scores + (1 - lambda_weights) * l2_slim_el_scores
linf_new_scores = lambda_weights * linf_rp3beta_scores + (1 - lambda_weights) * linf_slim_el_scores
lminusinf_new_scores = lambda_weights * lminusinf_rp3beta_scores + (1 - lambda_weights) * lminusinf_slim_el_scores

_ = plt.figure(figsize=(16,9))
ax = sns.scatterplot(data=l1_new_scores)
ax = sns.scatterplot(data=l2_new_scores) 
ax = sns.scatterplot(data=linf_new_scores) 
ax = sns.scatterplot(data=lminusinf_new_scores) 
ax.legend(["L1 Scores", "L2 Scores", "LInf Scores", "LMinusInf Scores"])

In [ ]:
from Recommenders.BaseRecommender import BaseRecommender
import scipy.sparse as sps

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            norm_item_weights_1 = 0.0000001
            #raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            norm_item_weights_2 = 0.0000001
            #raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights



In [ ]:
def objective(trial):
    norm = trial.suggest_categorical("norm", [1, 2, np.inf, -np.inf])

    recommender_object = DifferentLossScoresHybridRecommender(data_train, slim_el_recommender, rp3beta_recommender)
    recommender_object.fit(
        norm = norm, 
        alpha = 0.66)

    _, _, ev_map, _, _ = evaluator(recommender_object, data_train, data_val)
    
    return ev_map

for norm in [1, 2, np.inf, -np.inf]:
    rat_rank_study.optimize(objective, n_trials=1)

In [ ]:
rat_rank_study.best_params

In [ ]:
recommender_object = DifferentLossScoresHybridRecommender(data_train, slim_el_recommender, rp3beta_recommender)
recommender_object.fit(**rat_rank_study.best_params)

In [ ]:
submission2(recommender_object, users, usermap, itemmap, data_train)